# Семинар 5    
## Собираем поисковик 

![](https://bilimfili.com/wp-content/uploads/2017/06/bir-urune-emek-vermek-o-urune-olan-deger-algimizi-degistirir-mi-bilimfilicom.jpg) 


Мы уже все знаем, для того чтобы сделать поисковик. Осталось соединить все части вместе.    
Итак, для поисковика нам понадобятся:         
**1. База документов **
> в первом дз - корпус Друзей    
в сегодняшнем дз - корпус юридических вопросов-ответов    
в итоговом проекте - корпус Авито   

**2. Функция индексации**                 
Что делает: собирает информацию о корпусе, по которуму будет происходить поиск      
Своя для каждого поискового метода:       
> A. для обратного индекса она создает обратный индекс (чудо) и сохраняет статистики корпуса, необходимые для Okapi BM25 (средняя длина документа в коллекции, количество доков ... )             
> B. для поиска через word2vec эта функция создает вектор для каждого документа в коллекции путем, например, усреднения всех векторов коллекции       
> C. для поиска через doc2vec эта функция создает вектор для каждого документа               

   Не забывайте сохранить все, что насчитает эта функция. Если это будет происходить налету во время поиска, понятно, что он будет работать сто лет     
   
**3. Функция поиска**     
Можно разделить на две части:
1. функция вычисления близости между запросом и документом    
> 1. для индекса это Okapi BM25
> 2. для w2v и d2v это обычная косинусная близость между векторами          
2. ранжирование (или просто сортировка)


Время все это реализовать.

# Индексация
## Word2Vec
### Задание 1
Загрузите любую понравившуюся вам word2vec модель

In [16]:
from gensim.models import Word2Vec, KeyedVectors

In [17]:
model_path = 'D:\\work\\ir\\ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec'

In [18]:
# если модель с POS-тэггингом
model = KeyedVectors.load_word2vec_format(model_path, binary=False)

### Задание 2 
Напишите функцию индексации для поиска через word2vec. Она должна для каждого документа из корпуса строить вектор.   
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только  вектор, но и опознователь текста, которому он принадлежит. 
Для поисковика это может быть url страницы, для поиска по текстовому корпусу сам текст.

> В качестве документа для word2vec берите **параграфы** исходного текста, а не весь текст целиком. Так вектора будут более осмысленными. В противном случае можно получить один очень общий вектор, релевантый совершенно разным запросам.

In [82]:
import nltk
import pymorphy2
import os
import numpy as np

morph = pymorphy2.MorphAnalyzer()
main_dir = 'D:\\work\\ir\\small_corp'

In [63]:
files_list = []
filenames_list = []

for root, dirs, filenames in os.walk(main_dir):
    for filename in filenames:
        filepath = os.path.join(root, filename)
        filenames_list.append(os.path.splitext(filename)[0].split('-')[-1].strip())
        with open(filepath, 'r',encoding='utf-8') as txt_file:
            files_list.append(txt_file.read())     

In [70]:
files_list = [text.split('text:')[-1] for text in files_list]

In [71]:
from russian_tagsets import converters
to_ud = converters.converter('opencorpora-int', 'ud20')

In [72]:
tokens_list = []

for file in files_list:
    parsed_line = []
    tokens = nltk.word_tokenize(file)
    for token in tokens:
        res = morph.parse(token)[0]
        if res.tag.POS:
            new_token =  res.normal_form +'_'+ to_ud(res.tag.POS).split()[0]
            parsed_line.append(new_token)
    parsed_line = " ".join(parsed_line)
    tokens_list.append(parsed_line)

In [74]:
tokens_list[4]

'в_ADP хороший_ADJ состояние_NOUN 2007г_NOUN'

In [106]:
save_path = 'D:\\work\\ir\\db_w2v\\'

In [104]:
def get_w2v_vectors(tokens_string):
    """Получает вектор документа"""
    vec = np.zeros((300))
    counter = 0
    for token in tokens_string.split():
        if token in model:
            counter += 1 
            vec = np.add(vec, model[token])
    if counter > 0: 
        return vec/counter
    else:
        return vec

def save_w2v_base():
    """Индексирует всю базу для поиска через word2vec"""
    for idx, token_string in enumerate(tokens_list):
        np.save( save_path + filenames_list[idx] + '.npy', get_w2v_vectors(token_string))

In [107]:
save_w2v_base()

## Doc2Vec
### Задание 3
Напишите функцию обучения doc2vec на юридических текстах, и получите свою кастомную d2v модель. 
> Совет: есть мнение, что для обучения doc2vec модели не нужно удалять стоп-слова из корпуса. Они являются важными семантическими элементами.      

Важно! В качестве документа для doc2vec берите **параграфы** исходного текста, а не весь текст целиком. И не забывайте про предобработку.

In [161]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

d2v_save_path = 'D:\\work\\ir\\db_d2v\\'

In [142]:
docs = []


for idx, file in enumerate(files_list):
    parsed_line = []
    tokens = nltk.word_tokenize(file)
    for token in tokens:
        res = morph.parse(token)[0]
        if res.tag.POS:
            new_token =  res.normal_form +'_'+ to_ud(res.tag.POS).split()[0]
            parsed_line.append(new_token)
    t = TaggedDocument(parsed_line, [filenames_list[idx]])
    docs.append(t)



In [147]:
def train_doc2vec():
    tagged_data = docs
    d2model = Doc2Vec(vector_size=100, min_count=1, alpha=0.025, 
                min_alpha=0.025, epochs=100, workers=4, dm=1)
    d2model.build_vocab(tagged_data)
    d2model.train(tagged_data, total_examples=d2model.corpus_count, epochs=d2model.epochs)
    return d2model

In [149]:
d2v = train_doc2vec()


array([-1.4122753 ,  0.13451298, -0.6794814 ,  0.8887192 , -0.5297949 ,
       -1.5784425 ,  0.28120717, -0.8307996 , -0.834838  , -1.4230663 ,
       -0.07339069, -0.252388  ,  0.34186664,  1.6430672 ,  1.88629   ,
       -0.39578202,  1.3162977 , -2.6882977 , -0.04897378, -0.978478  ,
       -0.22896   , -0.32751045, -0.8776489 , -0.1450509 , -0.54065126,
        0.13139144, -0.02085602,  0.96709913, -1.1883047 , -0.40493727,
        0.508319  , -0.32163307,  0.355028  ,  1.4780113 , -0.26284963,
       -0.6874224 , -0.5674473 ,  0.17464145,  1.5861082 , -0.7705194 ,
       -0.3348891 ,  0.15415905, -0.16587476, -0.21483152,  2.048567  ,
       -0.7060491 ,  0.4809902 ,  0.8401899 , -0.46194327, -0.5742235 ,
       -0.30292916, -0.33116862,  1.7527803 , -0.17137058, -1.0662539 ,
       -0.60030836,  0.97545576, -0.22212784,  1.6950212 ,  0.56585586,
        0.827901  , -0.05274556,  0.9357327 , -0.08777424, -0.04241303,
        0.310094  , -1.2013171 , -1.219638  , -0.7906719 , -1.09

### Задание 4
Напишите функцию индексации для поиска через doc2vec. Она должна для каждого документа из корпуса получать вектор.    
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только вектор, но и опознователь текста, которому он принадлежит. 

In [164]:
def get_d2v_vectors(filename):
    """Получает вектор документа"""
    return d2v.docvecs[filename]

def save_d2v_base():
    """Индексирует всю базу для поиска через doc2vec"""
    for filename  in filenames_list:
        np.save( d2v_save_path + filename + '.npy', get_d2v_vectors(filename))
    

In [165]:
save_d2v_base()

# Функция поиска

Для обратного индекса функцией поиска является Okapi BM25. Она у вас уже должна быть реализована.

In [205]:
def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between word and document from collection
    :return: score
    """
    idf = log((N -n +0.5)/(n + 0.5)) 
    numerator = (k1 + 1) * qf
    denumerator = qf + k1* (1 - b + b  * (dl / avgdl))
    return idf * numerator / denumerator


def compute_sim(lemma, inverted_index, relevance_dict) -> float:
    """
    Compute similarity score between word in search query and all document  from collection
    :return: score
    """
    doc_freqs = inverted_index[lemma]
    n = doc_freqs.astype('bool').sum()
    N = len(doc_freqs.index)
    avgdl = length_series.mean()
    for doc in doc_freqs.index:
        qf = doc_freqs[doc]
        dl = length_series[doc]
        relevance_dict[doc] += score_BM25(qf, dl, avgdl, k1, b, N, n)
    return relevance_dict


def okapi_search(query) -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    relevance_dict = defaultdict(float)
    for word in query.split():
        relevance_dict = compute_sim(morph.parse(word)[0].normal_form, inverted_idx, relevance_dict)
    relevance_series = pd.Series(list(relevance_dict.values()))
    relevance_series.index = filenames_list
    sorted_relevance = relevance_series.sort_values(ascending=False)
    return sorted_relevance

Функция измерения близости между векторами нам пригодится:

In [166]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

In [181]:
from collections import defaultdict
import pandas as pd

### Задание 5
Напишите функцию для поиска через word2vec и для поиска через doc2vec, которая по входящему запросу выдает отсортированную выдачу документов.

In [201]:
def search_w2v(query):
    relevance_dict = defaultdict(float)
    words = query.split()
    for word in words:
        res = morph.parse(word)[0]
        if res.tag.POS:
            new_token =  res.normal_form +'_'+ to_ud(res.tag.POS).split()[0]
            relevance_dict = w2v_lemma_sim(new_token, relevance_dict)
    relevance_series = pd.Series(list(relevance_dict.values()))
    relevance_series.index = filenames_list
    sorted_relevance = relevance_series.sort_values(ascending=False)
    return sorted_relevance

def w2v_lemma_sim(lemma, relevance_dict):
    for filename in filenames_list:
        doc_vec = np.load(save_path + filename + '.npy')
        if lemma in model:
            lemma_vec = model[lemma]
        else:
            lemma_vec = np.zeros(300)
        relevance_dict[filename] += similarity(doc_vec, lemma_vec)
    return relevance_dict

def d2v_lemma_sim(lemma, relevance_dict):
    for filename in filenames_list:
        doc_vec = np.load(d2v_save_path + filename + '.npy')
        if lemma in d2v.wv:
            lemma_vec = d2v.wv[lemma]
        else:
            lemma_vec = np.zeros(100)
        relevance_dict[filename] += similarity(doc_vec, lemma_vec)
    return relevance_dict


    

def search_d2v(query):
    relevance_dict = defaultdict(float)
    words = query.split()
    for word in words:
        res = morph.parse(word)[0]
        if res.tag.POS:
            new_token =  res.normal_form +'_'+ to_ud(res.tag.POS).split()[0]
            relevance_dict = d2v_lemma_sim(new_token, relevance_dict)
    relevance_series = pd.Series(list(relevance_dict.values()))
    relevance_series.index = filenames_list
    sorted_relevance = relevance_series.sort_values(ascending=False)
    return sorted_relevance






После выполнения всех этих заданий ваш поисковик готов, поздравляю!                  
Осталось завернуть все написанное в питон скрипт, и сделать общую функцию поиска гибким, чтобы мы могли искать как по обратному индексу, так и по word2vec, так и по doc2vec.          
Сделать это можно очень просто через старый добрый ``` if ```, который будет дергать ту или иную функцию поиска:

In [ ]:
def search(query, search_method):
    if search_method == 'inverted_index':

        search_result = okapi_search()
    elif search_method == 'word2vec':
        ...
        search_result = search_w2v()
    elif search_method == 'doc2vec':
        ...
        search_result = search_d2v()
    else:
        raise TypeError('unsupported search method')